In [35]:

import os
import random

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import layers, models, datasets
from tensorflow.keras.applications import MobileNet
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.layers import Resizing
from tensorflow.keras.optimizers import Adam


def load_dataset(validation_split=0.2,dec_factor=10):
    # Load the CIFAR-10 dataset
    (train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()


    # Reduce the number of images by a factor of dec_factor
    train_images = train_images[::dec_factor]  # Take every Nth image
    train_labels = train_labels[::dec_factor]  # Corresponding labels
    test_images = test_images[::dec_factor]
    test_labels = test_labels[::dec_factor]


    # Split the training data into training and validation sets
    train_images, val_images, train_labels, val_labels = train_test_split(
        train_images, train_labels, test_size=validation_split, random_state=42)

    # Normalize pixel values to be between 0 and 1
    train_images, val_images, test_images = train_images / 255.0, val_images / 255.0, test_images / 255.0
    
    # Convert labels to one-hot encoding
    train_labels = to_categorical(train_labels, 10)
    val_labels = to_categorical(val_labels, 10)
    test_labels = to_categorical(test_labels, 10)

    return train_images, train_labels, val_images, val_labels, test_images, test_labels


def create_model():
    image_size = 128
    base_model = MobileNet( input_shape = (image_size,image_size,3),
                            include_top=False,
                             weights='imagenet')
    base_model.trainable = False  # Freeze the base model

    model = models.Sequential([        
        Resizing(image_size, image_size, interpolation="nearest", input_shape=train_images.shape[1:]),
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(10, activation='softmax')
    ])

    # Specify the learning rate
    
    # Instantiate the Adam optimizer with the default learning rate
    optimizer = Adam()

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


GPUs Available:  []
Is Metal enabled: []
TensorFlow version: 2.18.0


In [ ]:

# def plot_train_vs_val_accuracy(history):
#     plt.figure(figsize=(10,5))
#     plt.subplot(1,2,1)
#     t = np.array(history.epoch)
#     colors = {}
#     colors['loss'] = 'r'
#     colors['val_loss'] = '#FEA510'
#     colors['accuracy'] = 'green'
#     colors['val_accuracy'] = 'b'

#     # plt.figure(figsize=(10, 5))
#     i = 1
#     for elem in history.history.keys():
#         # i can check if the string contains a substring for subplot
#         if i%2 ==0:
#             plt.subplot(1,2,1)
#         else:
#             plt.subplot(1,2,2)

#         y = np.array(history.history[elem])
#         c = str(colors[elem])
#         l = str(elem)
#         plt.plot(np.array(t),np.array(y),color=c,label=l)
#         i += 1

#         plt.legend()
#         # plt.show()
#         plt.xlabel('Epochs')
#         plt.ylabel('Amplitude')
#     return


In [ ]:
# Set the random seeds
os.environ['PYTHONHASHSEED'] = str(42)  # This variable influences the hash function's behavior in Python 3.3 and later.
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

#Load the dataset
train_images, train_labels, val_images, val_labels, test_images, test_labels = load_dataset()
#Create the backbone model that will be used to train
model = create_model()
#Do the actual training
history = model.fit(train_images, train_labels, epochs=5, validation_data=(val_images, val_labels))
#Evaluate 
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"Test accuracy: {test_acc}, Test loss: {test_loss}")



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_train_vs_val_metrics(history, figsize=(8, 3)):
    """
    Plot training and validation metrics (loss and accuracy) from model training history.
    
    Parameters:
    -----------
    history : tensorflow.keras.callbacks.History
        The training history object returned by model.fit()
    figsize : tuple, optional
        Figure size as (width, height), default is (12, 5)
        
    Returns:
    --------
    fig : matplotlib.figure.Figure
        The created figure object containing both plots
    """
    # Define consistent styling
    METRICS_CONFIG = {
        'loss': {'color': 'red', 'subplot': 0, 'title': 'Model Loss'},
        'val_loss': {'color': '#FEA510', 'subplot': 0},
        'accuracy': {'color': 'green', 'subplot': 1, 'title': 'Model Accuracy'},
        'val_accuracy': {'color': 'blue', 'subplot': 1}
    }
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    axes = [ax1, ax2]
    
    epochs = np.array(history.epoch)
    
    # Plot each metric
    for metric, values in history.history.items():
        if metric in METRICS_CONFIG:
            config = METRICS_CONFIG[metric]
            subplot_idx = config['subplot']
            ax = axes[subplot_idx]
            
            ax.plot(epochs, values, 
                   color=config['color'],
                   label=metric.replace('_', ' ').title(),
                   linewidth=2)
            
            # Set titles and labels only once per subplot
            if 'title' in config:
                ax.set_title(config['title'])
                ax.set_ylabel(config['title'].split()[1])

            ax.set_xlabel('Epochs')
            ax.grid(True, linestyle='--', alpha=0.7)
            ax.legend()
    
    plt.tight_layout()
    # return fig
plot_train_vs_val_metrics(history, figsize=(12, 5))

# part 3

In [ ]:
# a # Unfreeze the layers (base_model.trainable) so that the whole network can learn

for layer in model.layers:
    layer.trainable=True

[setattr(layer, 'trainable', True) for layer in model.layers]

model.summary()


In [ ]:
model.optimizer.learning_rate = 0.01


In [ ]:
new_model = tf.keras.Sequential(model.layers)  # Copy existing layers
new_model.add(tf.keras.layers.Dropout(0.5))    # Add dropoutos.environ['TF_ENABLE_GRAPH_REWRITE'] = '0'
